# Lab 3: Genome-scale Metabolic Modeling
### Mengnan Shi, Muhammad Arif and Cheng Zhang - 2023

Taken from multiple sources

## 0. Lab Preparations

To begin with our lab, we have to install package for the GEM analysis. There are 2 main toolboxes:

1. RAVEN: Only available in MATLAB
2. COBRA: Available in MALTAB and python

For this lab, we will be using COBRApy, a branch of COBRA toolbox for Python. In order to do that, you have to install it:

```shell
pip install cobra==0.24.0
```

OR

```shell
conda install -c bioconda cobra=0.24.0
```

Afterwards, please install pytest, pytest_benchmark, escher and markupsafe (v2.0.1) (same way as above). Once you completed the setup, check your installation by running the codes below and make sure you don't encounter any error.

In [ ]:
import cobra
from cobra.test import test_all
import pytest
import pytest_benchmark

GEM_File = 'files/Ec_core_flux1.xml'

In [ ]:
#optional: to print the full dataframe please run the cell
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
cobra.Configuration()

Download the files required for the lab via (please extract the files in a folder called "files" right above this script): XXXXXX

## 1. Exploring the model

By starting the GEM based analysis, you will need to import the model into CobraPy. Most of the model in the field are saved in SBML format (.xml file). ‘iOD907.xml’ is a GEM of Kluyveromyces lactis.

In [ ]:
%%capture
# load the model. The capture magic fucntion above is just to suppress all outputs for this cell
model = cobra.io.read_sbml_model(GEM_File)

#### Q1: How many reactions, metabolites and genes are there in this model? How can you know more about the reactions, metabolites and genes using the information included in the model?

#### Q2: How can you know more about the reactions, metabolites and genes using the information included in the model? List 10 Reactions together with their ID and formula. (Hint: model.reactions)

#### Q3a: Explore reaction called "FBP" (programmatically). Explain about the reaction, including but not limited to the formula, the enzymes/genes, and their reversibility. (Hint: explore the functions/variables under model.reactions)

#### Q3b: Explore metabolites called "atp_c" (programmatically). Explain about the metabolites, including but not limited to the compartment and reactions related to this metabolite. (Hint: explore the functions/variables under model.metabolites)

#### Q4: We can use 'model.exchanges' to explore all the exchange reactions. We can also explore the exchange reactions the same way as the previous question. Take one of the exchange reaction and explore it same as Q3, compare the results.

## 2. Flux Balance Analysis

Most modelling approaches using GEMs are based on the mass balancing around the internal metabolites in the system. However, in the exchange reactions, there are metabolites that only appear once in the model which would block the reaction. These metabolites are kept in the model to help the user to understand the biological meaning of the reaction.

Now, the model is ready for simulation, and we will start with simple example. As discussed in the lecture, we need to constrain the exchange reaction of the model, and provide it with an objective function for simulation.

#### Q5: Find the objective function of the model and explain the components. Is it maximization or minimization of the function? (Hint: objective, objective_coefficients, objective_direction)

The problem can now be solved using linear programming. The ‘optimize’ function which uses the solver will take a model and solves the linear programming problem defined by the constraints and the objective value coefficients. For now, the default solver is "glpk"

In [ ]:
cobra.Configuration()

In [ ]:
solution = model.optimize()

Now, it's time to perform the Flux Balance Analysis (FBA). The obtained structure ‘solution’ now contains multiple informations, namely "fluxes", "reduced_cost", "objective_value", and "status" which represent the full solution (flux distribution) to the model, the value of objective function (only it’s absolute value is important in this case). The variable "solution.fluxes" contains the fluxes of all reactions, whereas "solution.status" shows the status of the optimization (optimal or infeasible) and "solution.objective_value" shows the flux of the objective functions (that were maximized or minimized). 

In [ ]:
solution.objective_value

In [ ]:
solution.fluxes

In [ ]:
solution.status

To be considered as carrying flux, a reaction has to have flux (in absolute value) above 10^-7 which is the accuracy of the LP solver. In other words, anything between -10^-7 to 10^-7 should be considered as 0).

In [ ]:
len(solution.fluxes[solution.fluxes.abs()>10e-7])

We can also summarize the solution into the the main findings/behavior of the optimization. We can also check the individual metabolite or reaction behaviours.

In [ ]:
model.summary()

In [ ]:
model.metabolites.nadh_c.summary()

In [ ]:
model.reactions.GAPD.summary()

Now, let's change the objective function of the model to the reaction of ATP Maintenance (ATPM). Don't forget to change the objective_coefficient of the biomass function to 0.

In [ ]:
model.reactions.ATPM.objective_coefficient=1
model.reactions.Biomass_Ecoli_core_N__w_GAM_.objective_coefficient=0

#### Q6: Can the new objective function optimized by the operation? How many reactions carried flux when ATP Maintenance? (Perform again all the steps above)

### 3. Flux Variability Analysis

In [ ]:
%%capture
# load the model. The capture magic fucntion above is just to suppress all outputs for this cell
model = cobra.io.read_sbml_model(GEM_File)

Then, we will work on flux variability analysis (FVA). Essentially, flux variability analysis is looping FBA to explore the achievable upper and lower bounds of each reaction under specific constraints. You can also set the "looselessness" of the optimality (fraction_of_optimum)

In [ ]:
fva=cobra.flux_analysis.flux_variability_analysis(model,fraction_of_optimum=0.9)

In [ ]:
fva

In [ ]:
#Finding essential reactions
fva[fva.abs()>10e-7].dropna().shape

In [ ]:
#Finding blocked reactions (below 10^-7 threshold)
fva[(fva.abs()<10e-7).sum(1) == 2].shape

In [ ]:
fva[(fva.abs()<10e-7).sum(1) == 2]

Now, let's change the objective function of the model to the reaction of ATP Maintenance (ATPM). Don't forget to change the objective_coefficient of the biomass function to 0.

In [ ]:
model.reactions.ATPM.objective_coefficient=1
model.reactions.Biomass_Ecoli_core_N__w_GAM_.objective_coefficient=0

#### Q7: How many essential and blocked reactions are there now with the new results? Is there more essential & blocked reactions compared to last result? Why?

### 4. Essentiality Analysis

Essentiality analysis is one of the most commonly used GEM based analysis, and it identifies the key reactions/genes/metabolites in the GEM that is required for the objective function. We will use the bacteria GEM again and perform essentiality analysis to identify what reactions and genes are essential for its growth.

In [ ]:
%%capture
# load the model. The capture magic fucntion above is just to suppress all outputs for this cell
model = cobra.io.read_sbml_model(GEM_File)

solution=model.optimize()
original_objective=solution.objective_value

In [ ]:
rxn_essentiality = cobra.flux_analysis.single_reaction_deletion(model)
rxn_essentiality

In [ ]:
fva[fva.abs()>10e-7].dropna().index

Here, we defined a reaction to be essential if its deletion will cause the growth decrease to less than 10% of its maximal value. Apparently, this is very strict criterion, and we can also try a different cutoff to see what will be changed.

In [ ]:
cutoff=0.1
EssentialReactions10_rxn=rxn_essentiality[(rxn_essentiality['growth'] <= cutoff*original_objective) | (rxn_essentiality['status'] == 'infeasible')]

In [ ]:
EssentialReactions10_rxn

Now we make the criterion to be 50% of the maximum value. As we can see, there will be more essential reactions if we set a higher cutoff.

In [ ]:
cutoff=0.5
EssentialReactions50_rxn=rxn_essentiality[(rxn_essentiality['growth'] <= cutoff*original_objective) | (rxn_essentiality['status'] == 'infeasible')]

#### Q8: How many reaction(s) is additional in EssentialReactions50 compared to EssentialReactions10? Can you list them?

On top of reaction essentiality analysis (that we performed above), we can also perform gene essentiality analysis. 

In [ ]:
gene_essentiality = cobra.flux_analysis.single_gene_deletion(model)
gene_essentiality

Same as essential reactions, we defined a gene to be essential if its deletion will cause the growth decrease to less than 10% of its maximal value. 

In [ ]:
EssentialReactions10_gene=gene_essentiality[(gene_essentiality['growth'] <= cutoff*original_objective) | (gene_essentiality['status'] == 'infeasible')]
EssentialReactions10_gene

### Visualization of GEM Analysis with Escher

One way to visualize the GEM Analysis is to use package called Escher. It can be used through Python or as web-based application. In this lab, we will focus our tasks to visualize the data via the python package.


If you encounter any problem, refer to their documentation: https://escher.readthedocs.io/en/latest/

In [ ]:
import cobra
from cobra.test import test_all
import escher
from escher import Builder
from time import sleep


GEM_File = 'files/Ec_core_flux1.xml'

In [ ]:
%%capture
# load the model. The capture magic fucntion above is just to suppress all outputs for this cell
model = cobra.io.read_sbml_model(GEM_File)

solution=model.optimize()

First, we have to build the basic GEM map. In this lab, we used E. Coli Core Model.

In [ ]:
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core',
)


Next, we can integrate our FBA results with the basic GEM Map (hint: the color bar can be checked by `Setting` under section `View`)

In [ ]:
builder.reaction_data = solution.fluxes
builder.metabolite_data = solution.shadow_prices
builder.reaction_scale_preset = 'GaBuRd'

In [ ]:
builder

We can also save it as HTML for further analysis

In [ ]:
builder.save_html('Map_EColi_BonusLab_Biomass.html')

## Bonus Questions

**Note**: To get full points in the **bonus part** you only have to answer 3 out of the 4 questions below, but you **have to specify** on your report which question you are leaving out.

#### B1: Change the objective function to ATPM. Perform the reaction and gene essentiality analysis of the model with 10% cutoff.

#### B2: Compare the result of the essentiality analysis with the results from the previous questions (when biomass was still the objective function). Any changes? Any new conclusion?

#### B3: Visualize the FBA result of the model (with ATPM as the objective function) with Escher. Save the result as HTML

#### B4: Compare the map between ATPM and biomass as objective function and point out the difference. (Hint: Use screenshots/picture to help you)